# Imports y ETL

In [1]:
import pandas as pd
import json

In [2]:
df_bixi14 = pd.read_csv('./OD_2014.csv')
df_bixi_station14 = pd.read_csv('./Stations_2014.csv')
df_bixi15 = pd.read_csv('./OD_2015.csv')
df_bixi_station15 = pd.read_csv('./Stations_2015.csv')
df_bixi2y = df_bixi14.append([df_bixi15] , ignore_index=True) #Join de años 2014 y 2015

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   code       919 non-null    int64  
 1   name       919 non-null    object 
 2   latitude   919 non-null    float64
 3   longitude  919 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 28.8+ KB


In [3]:
df_bixi2y.info()
df_bixi2y = df_bixi2y.drop(columns=["Unnamed: 0"])# Elimino la columna unnamed
# cambio a integer y datetime varias columnas
df_bixi2y['start_station_code'] = pd.to_numeric(df_bixi2y['start_station_code'] , errors='coerce' , downcast='integer')
df_bixi2y['end_station_code'] = pd.to_numeric(df_bixi2y['end_station_code'] , errors='coerce' , downcast='integer')
df_bixi2y['start_date'] = pd.to_datetime(df_bixi2y['start_date'])
df_bixi2y['end_date'] = pd.to_datetime(df_bixi2y['end_date'])
df_bixi2y = df_bixi2y.dropna() # drop de valores null
df_bixi2y.info()
df_bixi2y_stations['code'] = pd.to_numeric(df_bixi2y_stations['code'] , errors='coerce' , downcast='integer')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6580159 entries, 0 to 6580158
Data columns (total 7 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   Unnamed: 0          int64 
 1   start_date          object
 2   start_station_code  int64 
 3   end_date            object
 4   end_station_code    int64 
 5   duration_sec        int64 
 6   is_member           int64 
dtypes: int64(5), object(2)
memory usage: 351.4+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6580159 entries, 0 to 6580158
Data columns (total 6 columns):
 #   Column              Dtype         
---  ------              -----         
 0   start_date          datetime64[ns]
 1   start_station_code  int16         
 2   end_date            datetime64[ns]
 3   end_station_code    int16         
 4   duration_sec        int64         
 5   is_member           int64         
dtypes: datetime64[ns](2), int16(2), int64(2)
memory usage: 276.1 MB


In [4]:
#Añado nuevas columnas para poder agrupar y calcular agregados
df_bixi2y['year_month'] = df_bixi2y.start_date.dt.to_period('M') #año y mes
df_bixi2y['year'] = df_bixi2y.start_date.dt.year #año
df_bixi2y['month'] = df_bixi2y.start_date.dt.month #mes
df_bixi2y['day'] = df_bixi2y.start_date.dt.dayofweek #dia semana
df_bixi2y['hour'] = df_bixi2y.start_date.dt.time #hora

In [5]:
df_bixi2y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6580159 entries, 0 to 6580158
Data columns (total 11 columns):
 #   Column              Dtype         
---  ------              -----         
 0   start_date          datetime64[ns]
 1   start_station_code  int16         
 2   end_date            datetime64[ns]
 3   end_station_code    int16         
 4   duration_sec        int64         
 5   is_member           int64         
 6   year_month          period[M]     
 7   year                int64         
 8   month               int64         
 9   day                 int64         
 10  hour                object        
dtypes: datetime64[ns](2), int16(2), int64(5), object(1), period[M](1)
memory usage: 527.1+ MB


# Utilización del sistema

In [6]:
df_bixi2y.groupby(by=['year'])['start_station_code'].count() # cantidad de viajes totales

year
2014    3136276
2015    3443883
Name: start_station_code, dtype: int64

In [36]:
df_bixi2y.groupby(by=['year'])['duration_sec'].sum()/3600 # tiempo utilización sistema en h

Index(['start_date', 'start_station_code', 'end_date', 'end_station_code',
       'duration_sec', 'is_member', 'year_month', 'year', 'month', 'day',
       'hour'],
      dtype='object')

# Capacidad instalada total

In [18]:
f_stat = open(r'./stations.json') # abrimos el fichero json
stat_json = json.load(f_stat)
df_stats = pd.json_normalize(stat_json, record_path=['stations']) # lo normalizamos ya que tiene una lista anidada
f_stat.close() # cerramos el descriptor del fichero json
df_stats.head()

,id,s,n,st,b,su,m,lu,lc,bk,bl,la,lo,da,dx,ba,bx
0,1,HÃ´tel-de-Ville 2 (du Champs-de-Mars / Gosford),6001,1,False,False,False,1510061994757,1510070565576,False,False,45.509328,-73.554347,0,0,15,0
1,2,Ste-Catherine / DÃ©zÃ©ry,6002,1,False,False,False,1510065732418,1510070446412,False,False,45.539292,-73.541031,11,0,12,0
2,3,Clark / Evans,6003,1,False,False,False,1510069557891,1510070398986,False,False,45.511132,-73.567907,11,0,8,0
3,4,HÃ´tel-de-Ville (du Champs-de-Mars / Gosford),6004,1,False,False,False,1510069651843,1510070613361,False,False,45.509188,-73.554580,9,0,26,0
4,6,18e avenue / Rosemont,6006,1,False,False,False,1510062707620,1510070500075,False,False,45.557895,-73.576529,9,0,14,0


In [39]:
df_stats['ba'].sum() # Suma de todas las capacidades de las estaciones


4925